In [1]:
import polars as pl
import pandas as pd

In [11]:
df = pl.DataFrame({'a':[1, 1, 2, 2, 3, 3], 'b':[3, 2, 1, 1, 2, 3], 'c':[3, 2, 1, 1, 2, 3]})
print(df)
lista_1 = ["b"]
lista_2 = ["c"]

print(df.group_by("a").agg([[pl.col(col).sum() for col in lista_1],[pl.col(col).max() for col in lista_2]]))

shape: (6, 3)
┌─────┬─────┬─────┐
│ a   ┆ b   ┆ c   │
│ --- ┆ --- ┆ --- │
│ i64 ┆ i64 ┆ i64 │
╞═════╪═════╪═════╡
│ 1   ┆ 3   ┆ 3   │
│ 1   ┆ 2   ┆ 2   │
│ 2   ┆ 1   ┆ 1   │
│ 2   ┆ 1   ┆ 1   │
│ 3   ┆ 2   ┆ 2   │
│ 3   ┆ 3   ┆ 3   │
└─────┴─────┴─────┘


DuplicateError: column with name 'literal' has more than one occurrences

Error originated just after this operation:
DF ["a", "b", "c"]; PROJECT */3 COLUMNS; SELECTION: "None"

In [18]:
a_list = ['I am just a string']

df = pl.DataFrame({
    'col1': ['I am just a string', 'one more, but without the letters', 'we want, a, b, c,', 'Nothing here']
})

df = df.with_columns(pl.col('col1').str.contains_any(a_list).alias("ok"))
print(df)

shape: (4, 2)
┌───────────────────────────────────┬───────┐
│ col1                              ┆ ok    │
│ ---                               ┆ ---   │
│ str                               ┆ bool  │
╞═══════════════════════════════════╪═══════╡
│ I am just a string                ┆ true  │
│ one more, but without the letter… ┆ false │
│ we want, a, b, c,                 ┆ false │
│ Nothing here                      ┆ false │
└───────────────────────────────────┴───────┘


In [34]:
import pandas as pd 
import numpy as np 
import polars as pl

segments = [str(i) for i in range(1_000)]

# My real data is ~500m rows
nums = np.random.choice(segments, (1_000_00,10))

df = pd.DataFrame({'segments': [','.join(n) for n in nums]})

In [37]:
df2 = pl.from_pandas(df[['segments']])
df2 = df2.with_columns(pl.col('segments').str.split(','))
print(df2.dtypes)
df_ans = (df2.with_columns([
              pl.arange(0, len(df2)).alias('row_index'), 
              pl.lit(1).alias('__one__')
          ])
           .explode('segments')
            .pivot(index='row_index', columns='segments', values='__one__', aggregate_function=pl.count())
            .fill_null(0))
     
df_one_hot_encoded = df_ans.to_pandas()

[List(String)]


C:\Users\Jan\AppData\Local\Temp\ipykernel_20448\3229931542.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .pivot(index='row_index', columns='segments', values='__one__', aggregate_function=pl.count())


In [38]:
df_one_hot_encoded

,row_index,543,765,810,279,41,136,83,376,835,...,685,904,244,103,628,524,948,55,310,822
0,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,99996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,99997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99998,99998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df2.dtypes

[String]